# Creating bedgraph files for different genomic/epigenomic features NPB

# WGD-derived duplicated genes
Duplicated genes dataset was downloaded from https://github.com/almeidasilvaf/polyploid_GRNs/tree/master<br>
Publication https://doi.org/10.1093/molbev/msad141

In [ ]:
#creating tsv file with info about each WGD/SSD derived duplicated gene (chr, start, end, name)

In [1]:
!head /scratch/ak8725/annotations/duplicated_genes.tsv

gene	type	peak
Os01g0136850	WGD	1
Os01g0169900	WGD	1
Os01g0179300	WGD	1
Os01g0180300	WGD	1
Os01g0200700	WGD	1
Os01g0218200	WGD	1
Os01g0231000	WGD	1
Os01g0285300	WGD	1
Os01g0292200	WGD	1


In [26]:
!head  /scratch/ak8725/annotations/NPB_genes.gff

chr01	irgsp1_locus	gene	2983	10815	.	+	.	ID=Os01g0100100;Name=Os01g0100100;Note=RabGAP/TBC domain containing protein. (Os01t0100100-01);Transcript variants=Os01t0100100-01
chr01	irgsp1_locus	gene	11218	12435	.	+	.	ID=Os01g0100200;Name=Os01g0100200;Note=Conserved hypothetical protein. (Os01t0100200-01);Transcript variants=Os01t0100200-01
chr01	irgsp1_locus	gene	11372	12284	.	-	.	ID=Os01g0100300;Name=Os01g0100300;Note=Cytochrome P450 domain containing protein. (Os01t0100300-00);Transcript variants=Os01t0100300-00
chr01	irgsp1_locus	gene	12721	15685	.	+	.	ID=Os01g0100400;Name=Os01g0100400;Note=Similar to Pectinesterase-like protein. (Os01t0100400-01);Transcript variants=Os01t0100400-01
chr01	irgsp1_locus	gene	12808	13978	.	-	.	ID=Os01g0100466;Name=Os01g0100466;Note=Hypothetical protein. (Os01t0100466-00);Transcript variants=Os01t0100466-00
chr01	irgsp1_locus	gene	16399	20144	.	+	.	ID=Os01g0100500;Name=Os01g0100500;Note=Immunoglobulin-like domain containing protein. (Os01t0100500-01);Trans

In [28]:
!awk -F'\t' 'BEGIN {OFS="\t"} {split($9, a, ";"); split(a[1], b, "="); print $1, $4, $5, b[2]}' /scratch/ak8725/annotations/NPB_genes.gff > IRGSP_genes.bed

In [29]:
!head /scratch/ak8725/annotations/IRGSP_genes.bed

chr01	2983	10815	Os01g0100100
chr01	11218	12435	Os01g0100200
chr01	11372	12284	Os01g0100300
chr01	12721	15685	Os01g0100400
chr01	12808	13978	Os01g0100466
chr01	16399	20144	Os01g0100500
chr01	22841	26892	Os01g0100600
chr01	25861	26424	Os01g0100650
chr01	27143	28644	Os01g0100700
chr01	29818	34453	Os01g0100800


In [30]:
!wc -l /scratch/ak8725/annotations/IRGSP_genes.bed

37828 /scratch/ak8725/annotations/IRGSP_genes.bed


In [4]:
awk 'NR>1 {print $1}' /scratch/ak8725/annotations/duplicated_genes.tsv | sort | uniq > gene_names.txt

join -1 1 -2 4 -o 2.1,2.2,2.3,1.1,1.2 -t $'\t' <(sort /scratch/ak8725/annotations/duplicated_genes.tsv) <(sort -t$'\t' -k4,4 /scratch/ak8725/annotations/IRGSP_genes.bed) > /scratch/ak8725/annotations/modified_duplicated_genes.tsv

sed -i '1i chr\tstart\tend\tgene_name\ttype' /scratch/ak8725/annotations/modified_duplicated_genes.tsv

In [5]:
mv /scratch/ak8725/annotations/modified_duplicated_genes.tsv /scratch/ak8725/annotations/duplicated_genes.tsv

In [10]:
#separate files for SSD and WGD genes
awk -F'\t' '$5 == "WGD" {OFS="\t"; print $1, $2, $3, $4}' /scratch/ak8725/annotations/duplicated_genes.tsv > /scratch/ak8725/annotations/WGD_genes.tsv
awk -F'\t' '$5 == "SSD" {OFS="\t"; print $1, $2, $3, $4}' /scratch/ak8725/annotations/duplicated_genes.tsv > /scratch/ak8725/annotations/SSD_genes.tsv

In [14]:
cut -f 1,2,3 /scratch/ak8725/annotations/SSD_genes.tsv > SSD_genes.bed
cut -f 1,2,3 /scratch/ak8725/annotations/WGD_genes.tsv > WGD_genes.bed

In [15]:
head SSD_genes.bed

chr01	2983	10815
chr01	12721	15685
chr01	29818	34453
chr01	58658	61090
chr01	82426	84095
chr01	89763	91521
chr01	139826	141555
chr01	148085	150568
chr01	151409	156449
chr01	170798	173144


In [29]:
mv WGD_genes.bed /scratch/ak8725/npb_feature_tracks

# Gene coverage per 500bp

In [3]:
awk 'BEGIN {OFS="\t"} {print $1, $4, $5}' NPB_genes.gff > NPB_genes.bed

In [5]:
module load bedtools/intel/2.29.2

In [10]:
bedtools makewindows -b /scratch/ak8725/NPB_new/map_generation/NPB_chr_coords.genome -w 500 > NPB_500bp_windows.bed

In [17]:
head NPB_genes.bed

chr01	2983	10815
chr01	11218	12435
chr01	11372	12284
chr01	12721	15685
chr01	12808	13978
chr01	16399	20144
chr01	22841	26892
chr01	25861	26424
chr01	27143	28644
chr01	29818	34453


In [18]:
bedtools intersect -a NPB_500bp_windows.bed -b NPB_genes.bed -wao > NPB_genes_in_windows.bed

***** WARNING: File NPB_genes.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	44902	45692

***** WARNING: File NPB_genes.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	44902	45692



In [26]:
head NPB_genes_in_windows.bed

chr01	0	500	.	-1	-1	0
chr01	500	1000	.	-1	-1	0
chr01	1000	1500	.	-1	-1	0
chr01	1500	2000	.	-1	-1	0
chr01	2000	2500	.	-1	-1	0
chr01	2500	3000	chr01	2983	10815	17
chr01	3000	3500	chr01	2983	10815	500
chr01	3500	4000	chr01	2983	10815	500
chr01	4000	4500	chr01	2983	10815	500
chr01	4500	5000	chr01	2983	10815	500


In [14]:
cut -f 1-3,10 NPB_te_in_windows.bed >  NPB_te_in_windows1.bed

In [22]:
bedtools groupby -g 1,2,3 -c 7 -o sum -i NPB_genes_in_windows.bed > npb_genes_in_windows_grouped.bed

In [29]:
head npb_genes_in_windows_fr.bed

chr01 0 500 0
chr01 500 1000 0
chr01 1000 1500 0
chr01 1500 2000 0
chr01 2000 2500 0
chr01 2500 3000 0.034
chr01 3000 3500 1
chr01 3500 4000 1
chr01 4000 4500 1
chr01 4500 5000 1


In [28]:
awk '{ $4 = $4 / 500; print }' npb_genes_in_windows_grouped.bed > npb_genes_in_windows_fr.bed

In [30]:
mv npb_genes_in_windows_fr.bed npb_genes_per_500bp.bedgraph

In [31]:
mv npb_genes_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

# WGD/SSD gene coverage per 500bp

In [5]:
module load bedtools/intel/2.29.2

In [43]:
head /scratch/ak8725/npb_feature_tracks/SSD_genes.bed

chr01	2983	10815
chr01	12721	15685
chr01	29818	34453
chr01	58658	61090
chr01	82426	84095
chr01	89763	91521
chr01	139826	141555
chr01	148085	150568
chr01	151409	156449
chr01	170798	173144


In [37]:
bedtools intersect -a NPB_500bp_windows.bed -b /scratch/ak8725/npb_feature_tracks/WGD_genes.bed -wao > WGD_genes_in_windows.bed
bedtools intersect -a NPB_500bp_windows.bed -b /scratch/ak8725/npb_feature_tracks/SSD_genes.bed -wao > SSD_genes_in_windows.bed

***** WARNING: File /scratch/ak8725/npb_feature_tracks/WGD_genes.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	655625	658866

***** WARNING: File /scratch/ak8725/npb_feature_tracks/WGD_genes.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	655625	658866

***** WARNING: File /scratch/ak8725/npb_feature_tracks/SSD_genes.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	100437	103373

***** WARNING: File /scratch/ak8725/npb_feature_tracks/SSD_genes.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	100437	103373



In [38]:
head SSD_genes_in_windows.bed

chr01	0	500	.	-1	-1	0
chr01	500	1000	.	-1	-1	0
chr01	1000	1500	.	-1	-1	0
chr01	1500	2000	.	-1	-1	0
chr01	2000	2500	.	-1	-1	0
chr01	2500	3000	chr01	2983	10815	17
chr01	3000	3500	chr01	2983	10815	500
chr01	3500	4000	chr01	2983	10815	500
chr01	4000	4500	chr01	2983	10815	500
chr01	4500	5000	chr01	2983	10815	500


In [39]:
bedtools groupby -g 1,2,3 -c 7 -o sum -i WGD_genes_in_windows.bed > wgd_genes_in_windows_grouped.bed
bedtools groupby -g 1,2,3 -c 7 -o sum -i SSD_genes_in_windows.bed > ssd_genes_in_windows_grouped.bed

In [42]:
head ssd_genes_in_windows_fr.bed

chr01 0 500 0
chr01 500 1000 0
chr01 1000 1500 0
chr01 1500 2000 0
chr01 2000 2500 0
chr01 2500 3000 0.034
chr01 3000 3500 1
chr01 3500 4000 1
chr01 4000 4500 1
chr01 4500 5000 1


In [41]:
awk '{ $4 = $4 / 500; print }' wgd_genes_in_windows_grouped.bed > wgd_genes_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' ssd_genes_in_windows_grouped.bed > ssd_genes_in_windows_fr.bed

In [44]:
mv wgd_genes_in_windows_fr.bed wgd_genes_per_500bp.bedgraph
mv ssd_genes_in_windows_fr.bed ssd_genes_per_500bp.bedgraph

In [45]:
mv wgd_genes_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv ssd_genes_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

# TIPs per 500bp

In [55]:
head /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs.bed

chr01	28721	28993	Os0018_Wanderer;MITE/Tourist	Deletion	+
chr01	29279	29541	Os0297;DNAnona/MULE	Deletion	-
chr01	30291	30685	Os0348;DNAnona/MULE	Deletion	-
chr01	34521	34753	Os3327;MITE/Stow	Deletion	+
chr01	36249	36255	Os0086;DNAnona/MULE	Insertion	+
chr01	41392	42226	RufipogonTE_00004098;DNA/DTM	Deletion	+
chr01	43170	58357	Os0030_INT_RIRE2-int;LTR/Gypsy	Deletion	-
chr01	135647	137462	Os0310_LTR;LTR/Gypsy	Deletion	-
chr01	149128	149133	Os2066_LTR;LTR/TRIM	Insertion	-
chr01	156633	164481	Os0988_INT-int;LTR/Copia	Deletion	-


In [73]:
wc -l /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs.bed
wc -l /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del.bed
wc -l /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins.bed

8392 /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs.bed
6100 /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del.bed
2292 /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins.bed


In [79]:
cut -f 1,2,3 /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del.bed > /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del_coords.bed
cut -f 1,2,3 /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins.bed > /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins_coords.bed
cut -f 1,2,3 /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs.bed > /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_coords.bed

In [80]:
bedtools intersect -a NPB_500bp_windows.bed -b /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del_coords.bed -wao > tips_del_in_windows.bed
bedtools intersect -a NPB_500bp_windows.bed -b /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins_coords.bed -wao > tips_ins_in_windows.bed
bedtools intersect -a NPB_500bp_windows.bed -b /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_coords.bed -wao > tips_in_windows.bed

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del_coords.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	23315	38327

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_del_coords.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	23315	38327

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins_coords.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	99974	99975

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_ins_coords.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	99974	99975

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/SV_TIPs/Omer_TIPs_coords.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	23

In [81]:
bedtools groupby -g 1,2,3 -c 7 -o sum -i tips_del_in_windows.bed > tips_del_in_windows_grouped.bed
bedtools groupby -g 1,2,3 -c 7 -o sum -i tips_ins_in_windows.bed > tips_ins_in_windows_grouped.bed
bedtools groupby -g 1,2,3 -c 7 -o sum -i tips_in_windows.bed > tips_in_windows_grouped.bed

In [82]:
awk '{ $4 = $4 / 500; print }' tips_del_in_windows_grouped.bed > omer_tips_del_per_500bp.bedgraph
awk '{ $4 = $4 / 500; print }' tips_ins_in_windows_grouped.bed > omer_tips_ins_per_500bp.bedgraph
awk '{ $4 = $4 / 500; print }' tips_in_windows_grouped.bed > omer_tips_per_500bp.bedgraph

In [83]:
mv omer_tips_del_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv omer_tips_ins_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv omer_tips_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

# GC content per 500bp

In [33]:
head /scratch/ak8725/NPB_new/NPB.genome

chr01	43270923
chr02	35937250
chr03	36413819
chr04	35502694
chr05	29958434
chr06	31248787
chr07	29697621
chr08	28443022
chr09	23012720
chr10	23207287


In [36]:
samtools faidx /scratch/ak8725/NPB_new/NPB.fna

In [42]:
bedtools makewindows -g /scratch/ak8725/NPB_new/NPB.genome -w 500 | bedtools nuc -fi /scratch/ak8725/NPB_new/NPB.fna -bed - | cut -f 1,2,3,5 > GC_content.bed

In [11]:
mv /scratch/ak8725/ipynb\ scripts\ and\ plots/npb_GC_content_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

# SV breakpoint density per 500bp

from SNP-Seek 3K RG Large Structural Variants release 1.0<br>
https://snp-seek.irri.org/download.zul<br>
Used del, dup, ins, inv<br>
For deletions, start and end coordinates were used as breakpoints. For insertions, start coordinates were used as breakpoints.

I made separate .bedgraph files for breakpoints DEL, DUP, INS, INV<br>
Then combined them using bedtools unionbedg

In [2]:
cd /scratch/ak8725/SV_from_SNP_Seek

In [5]:
ls

NB_DEL_mergesam_clustered.txt  azucena_dup_in_windows1.bed
NB_DUP_mergesam_clustered.txt  azucena_ins.bedgraph
NB_INS_mergesam_clustered.txt  azucena_ins_in_windows.bed
NB_INV_mergesam_clustered.txt  azucena_ins_in_windows1.bed
az_del.bed		       azucena_inv.bedgraph
az_dup.bed		       azucena_inv_in_windows.bed
az_ins.bed		       azucena_inv_in_windows1.bed
az_inv.bed		       intersect.sh
azucena_SV.bedgraph	       npb_del.bed
azucena_del.bedgraph	       npb_dup.bed
azucena_del_in_windows.bed     npb_ins.bed
azucena_del_in_windows1.bed    npb_inv.bed
azucena_dup.bedgraph	       slurm-32747895.out
azucena_dup_in_windows.bed


In [3]:
module load bedtools/intel/2.29.2

In [12]:
bedtools makewindows -b /scratch/ak8725/NPB_new/NPB_chr_coords.genome -w 500 > NPB_500bp_windows.bed

In [9]:
head npb_del.bed

chr01	1045	1046
chr01	1057	1058
chr01	1045	1046
chr01	1057	1058
chr01	1045	1046
chr01	1057	1058
chr01	1058	1059
chr01	1077	1078
chr01	1058	1059
chr01	1091	1092


In [23]:
# ran intersect.sh
# bedtools intersect -a NPB_500bp_windows.bed -b npb_del.bed -wao > NPB_del_in_windows.bed
# bedtools intersect -a NPB_500bp_windows.bed -b npb_dup.bed -wao > NPB_dup_in_windows.bed
# bedtools intersect -a NPB_500bp_windows.bed -b npb_ins.bed -wao > NPB_ins_in_windows.bed
# bedtools intersect -a NPB_500bp_windows.bed -b npb_inv.bed -wao > NPB_inv_in_windows.bed

In [4]:
head NPB_del_in_windows.bed

chr01	0	500	.	-1	-1	0
chr01	500	1000	.	-1	-1	0
chr01	1000	1500	chr01	1045	1046	1
chr01	1000	1500	chr01	1057	1058	1
chr01	1000	1500	chr01	1045	1046	1
chr01	1000	1500	chr01	1057	1058	1
chr01	1000	1500	chr01	1045	1046	1
chr01	1000	1500	chr01	1057	1058	1
chr01	1000	1500	chr01	1058	1059	1
chr01	1000	1500	chr01	1077	1078	1


In [11]:
cut -f 1-3,7 NPB_del_in_windows.bed >  NPB_del_in_windows1.bed
cut -f 1-3,7 NPB_dup_in_windows.bed >  NPB_dup_in_windows1.bed
cut -f 1-3,7 NPB_ins_in_windows.bed >  NPB_ins_in_windows1.bed
cut -f 1-3,7 NPB_inv_in_windows.bed >  NPB_inv_in_windows1.bed

In [12]:
head NPB_del_in_windows1.bed

chr01	0	500	0
chr01	500	1000	0
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1


In [13]:
bedtools groupby -g 1,2,3 -c 4 -o sum -i NPB_del_in_windows1.bed > npb_del_in_windows_grouped.bed
bedtools groupby -g 1,2,3 -c 4 -o sum -i NPB_dup_in_windows1.bed > npb_dup_in_windows_grouped.bed
bedtools groupby -g 1,2,3 -c 4 -o sum -i NPB_ins_in_windows1.bed > npb_ins_in_windows_grouped.bed
bedtools groupby -g 1,2,3 -c 4 -o sum -i NPB_inv_in_windows1.bed > npb_inv_in_windows_grouped.bed

In [47]:
head npb_del_per_500bp.bedgraph

chr01	0	500	0
chr01	500	1000	0
chr01	1000	1500	0.164
chr01	1500	2000	0
chr01	2000	2500	0
chr01	2500	3000	0.004
chr01	3000	3500	0
chr01	3500	4000	0
chr01	4000	4500	0
chr01	4500	5000	0


In [44]:
awk '{ $4 = $4 / 500; print }' npb_del_in_windows_grouped.bed > npb_del_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' npb_dup_in_windows_grouped.bed > npb_dup_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' npb_ins_in_windows_grouped.bed > npb_ins_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' npb_inv_in_windows_grouped.bed > npb_inv_in_windows_fr.bed

In [45]:
mv npb_del_in_windows_fr.bed npb_del_per_500bp.bedgraph
mv npb_dup_in_windows_fr.bed npb_dup_per_500bp.bedgraph
mv npb_ins_in_windows_fr.bed npb_ins_per_500bp.bedgraph
mv npb_inv_in_windows_fr.bed npb_inv_per_500bp.bedgraph

In [48]:
sed -i -e 's/\s/\t/g' npb_del_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_dup_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_ins_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_inv_per_500bp.bedgraph

In [33]:
mv npb_del_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv npb_dup_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv npb_ins_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv npb_inv_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

In [49]:
#group all SV breakpoints
bedtools unionbedg -i npb_del_per_500bp.bedgraph npb_ins_per_500bp.bedgraph npb_inv_per_500bp.bedgraph npb_dup_per_500bp.bedgraph -filler 0 -scores sum > npb_sv_per_500bp.bedgraph

In [51]:
awk '{print $1"\t"$2"\t"$3"\t"$4+$5+$6+$7}' npb_sv_per_500bp.bedgraph > tmp_file && mv tmp_file npb_sv_per_500bp.bedgraph

In [53]:
mv npb_sv_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

In [52]:
head npb_sv_per_500bp.bedgraph

chr01	0	500	0
chr01	500	1000	0
chr01	1000	1500	0.39
chr01	1500	2000	0.002
chr01	2000	2500	0
chr01	2500	3000	0.118
chr01	3000	3500	0
chr01	3500	4000	0
chr01	4000	4500	0.008
chr01	4500	5000	0


# SNP per 500bp

3K RG 29mio biallelic SNPs Dataset from SNP-Seek<br>
Downloaded bim file

In [22]:
!wget https://s3.amazonaws.com/3kricegenome/reduced/NB_final_snp.bim.gz

--2023-09-28 13:17:51--  https://s3.amazonaws.com/3kricegenome/reduced/NB_final_snp.bim.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.172.144, 52.216.130.5, 16.182.98.168, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.172.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182247184 (174M) [binary/octet-stream]
Saving to: ‘NB_final_snp.bim.gz’

NB_final_snp.bim.gz 100%[===================>] 173.80M  12.6MB/s    in 13s     

2023-09-28 13:18:04 (13.7 MB/s) - ‘NB_final_snp.bim.gz’ saved [182247184/182247184]



In [28]:
!tail NB_final_snp.bed

chr12	27530771	27530771
chr12	27530774	27530774
chr12	27530798	27530798
chr12	27530804	27530804
chr12	27530805	27530805
chr12	27530814	27530814
chr12	27530827	27530827
chr12	27530831	27530831
chr12	27530844	27530844
chr12	27530848	27530848


In [26]:
# Open the input .bim file
with open('NB_final_snp.bim', 'r') as input_file:
    # Open the output file for writing
    with open('NB_final_snp.bed', 'w') as output_file:
        # Iterate through each line in the input file
        for line in input_file:
            # Split the line into columns based on whitespace
            columns = line.strip().split()
            # Extract the values from the columns
            col1_value = int(columns[0])
            
            # Handle values of 10, 11, and 12 in col1
            if col1_value >= 10:
                col1 = 'chr' + str(col1_value)
            else:
                col1 = 'chr' + str(col1_value).zfill(2)  # Format as 'chr01', 'chr02', etc.
            
            col2 = columns[3]
            col3 = columns[3]
            # Write the formatted columns to the output file
            output_file.write(f'{col1}\t{col2}\t{col3}\n')

# Print a message to confirm the operation
print('Output file created successfully.')

Output file created successfully.


In [30]:
!awk '{ $3 = $3 + 1; print }' NB_final_snp.bed > temp.bed && mv temp.bed NB_final_snp.bed

In [32]:
#making file tab-delimited
!sed -i -e 's/\s/\t/g' NB_final_snp.bed

In [33]:
!head NPB_snp_in_windows1.bed

chr01	0	500	0
chr01	500	1000	0
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1
chr01	1000	1500	1


In [60]:
#ran intersect.sh
# bedtools intersect -a NPB_500bp_windows.bed -b NB_final_snp.bed -wao > NPB_snp_in_windows.bed

In [4]:
cut -f 1-3,7 NPB_snp_in_windows.bed >  NPB_snp_in_windows1.bed
bedtools groupby -g 1,2,3 -c 4 -o sum -i NPB_snp_in_windows1.bed > npb_snp_in_windows_grouped.bed
awk '{ $4 = $4 / 500; print }' npb_snp_in_windows_grouped.bed > npb_snp_in_windows_fr.bed
mv npb_snp_in_windows_fr.bed npb_snp_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_snp_per_500bp.bedgraph

In [7]:
mv npb_snp_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

In [6]:
tail npb_snp_per_500bp.bedgraph

chr12	27527000	27527500	0.01
chr12	27527500	27528000	0.032
chr12	27528000	27528500	0.028
chr12	27528500	27529000	0.068
chr12	27529000	27529500	0.08
chr12	27529500	27530000	0.046
chr12	27530000	27530500	0.114
chr12	27530500	27531000	0.1
chr12	27531000	27531500	0
chr12	27531500	27531856	0


In [69]:
bedtools groupby -g 1,2,3 -c 4 -o sum -i NPB_snp_in_windows1.bed > npb_snp_in_windows_grouped.bed

In [71]:
head npb_snp_in_windows_grouped.bed

chr01	0	500	0
chr01	500	1000	0
chr01	1000	1500	0
chr01	1500	2000	-1
chr01	2000	2500	-1
chr01	2500	3000	0
chr01	3000	3500	0
chr01	3500	4000	0
chr01	4000	4500	0
chr01	4500	5000	0


In [44]:
awk '{ $4 = $4 / 500; print }' npb_del_in_windows_grouped.bed > npb_del_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' npb_dup_in_windows_grouped.bed > npb_dup_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' npb_ins_in_windows_grouped.bed > npb_ins_in_windows_fr.bed
awk '{ $4 = $4 / 500; print }' npb_inv_in_windows_grouped.bed > npb_inv_in_windows_fr.bed

In [45]:
mv npb_del_in_windows_fr.bed npb_del_per_500bp.bedgraph
mv npb_dup_in_windows_fr.bed npb_dup_per_500bp.bedgraph
mv npb_ins_in_windows_fr.bed npb_ins_per_500bp.bedgraph
mv npb_inv_in_windows_fr.bed npb_inv_per_500bp.bedgraph

In [48]:
sed -i -e 's/\s/\t/g' npb_del_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_dup_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_ins_per_500bp.bedgraph
sed -i -e 's/\s/\t/g' npb_inv_per_500bp.bedgraph

In [33]:
mv npb_del_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv npb_dup_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv npb_ins_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks
mv npb_inv_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

In [49]:
#group all SV breakpoints
bedtools unionbedg -i npb_del_per_500bp.bedgraph npb_ins_per_500bp.bedgraph npb_inv_per_500bp.bedgraph npb_dup_per_500bp.bedgraph -filler 0 -scores sum > npb_sv_per_500bp.bedgraph

In [51]:
awk '{print $1"\t"$2"\t"$3"\t"$4+$5+$6+$7}' npb_sv_per_500bp.bedgraph > tmp_file && mv tmp_file npb_sv_per_500bp.bedgraph

In [53]:
mv npb_sv_per_500bp.bedgraph /scratch/ak8725/npb_feature_tracks

In [ ]:
NPB_snp_in_windows.bed

# NPB TAD coverage per 500bp

In [4]:
module load bedtools/intel/2.29.2

In [7]:
head ../npb_feature_tracks/NPB_TADs_5kb_two_tools_80.bed

chr01	150000	210000
chr01	370000	420000
chr01	420000	525000
chr01	525000	560000
chr01	1675000	1765000
chr01	2000000	2055000
chr01	2590000	2640000
chr01	2640000	2695000
chr01	2695000	2810000
chr01	2810000	2840000


In [11]:
cd ../npb_feature_tracks
bedtools intersect -a NPB_500bp_windows.bed -b NPB_TADs_5kb_two_tools_80.bed -wao > tads_in_windows.bed

***** WARNING: File NPB_TADs_5kb_two_tools_80.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	350000	385000

***** WARNING: File NPB_TADs_5kb_two_tools_80.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	350000	385000



In [12]:
head tads_in_windows.bed

chr01	0	500	.	-1	-1	0
chr01	500	1000	.	-1	-1	0
chr01	1000	1500	.	-1	-1	0
chr01	1500	2000	.	-1	-1	0
chr01	2000	2500	.	-1	-1	0
chr01	2500	3000	.	-1	-1	0
chr01	3000	3500	.	-1	-1	0
chr01	3500	4000	.	-1	-1	0
chr01	4000	4500	.	-1	-1	0
chr01	4500	5000	.	-1	-1	0


In [13]:
bedtools groupby -g 1,2,3 -c 7 -o sum -i tads_in_windows.bed > tads_in_windows_grouped.bed

In [15]:
head tads_in_windows_fr.bed

chr01 0 500 0
chr01 500 1000 0
chr01 1000 1500 0
chr01 1500 2000 0
chr01 2000 2500 0
chr01 2500 3000 0
chr01 3000 3500 0
chr01 3500 4000 0
chr01 4000 4500 0
chr01 4500 5000 0


In [14]:
awk '{ $4 = $4 / 500; print }' tads_in_windows_grouped.bed > tads_in_windows_fr.bed

In [17]:
mv tads_in_windows_fr.bed tads_per_500bp.bedgraph

In [3]:
#making file tab-delimited
cd ../npb_feature_tracks
sed -i -e 's/\s/\t/g' tads_per_500bp.bedgraph

# Calculating sequnce identity between regions of the two modes

In [2]:
#creating bedpe file with mode1/2 regions from NPB and Omer corresponding
# Define input files
mode1_bed_file = "/scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/right.bed"
npb_omer_pairs_bedpe_file = "/scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/NPB-Oruf_pairs.bedpe"
output_bedpe_file = "/scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/right.bedpe"

# Create a dictionary to store NPB-Omer pairs by region
npb_omer_pairs = {}

# Read NPB-Omer pairs into the dictionary
with open(npb_omer_pairs_bedpe_file, "r") as npb_omer_pairs_bedpe_fh:
    for line in npb_omer_pairs_bedpe_fh:
        fields = line.strip().split("\t")
        key = tuple(fields[0:3])
        value = fields[3:6]
        npb_omer_pairs[key] = value

# Open the output bedpe file for writing
with open(output_bedpe_file, "w") as output_fh:
    # Read mode1.bed and write the new bedpe file
    with open(mode1_bed_file, "r") as mode1_bed_fh:
        for line in mode1_bed_fh:
            fields = line.strip().split("\t")
            region = tuple(fields)
            if region in npb_omer_pairs:
                npb_omer_values = npb_omer_pairs[region]
                output_fh.write("\t".join(fields) + "\t" + "\t".join(npb_omer_values) + "\n")

print(f"New bedpe file {output_bedpe_file} created.")


New bedpe file /scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/right.bedpe created.


In [104]:
#filtered out regions >=1Mb, see Filtering regions.ipynb
wc -l /scratch/ak8725/chess/whole_lifted/500kb_250kb_25/NPB-Omer/mode1_filtered.bedpe
wc -l /scratch/ak8725/chess/whole_lifted/500kb_250kb_25/NPB-Omer/mode2_filtered.bedpe

16 /scratch/ak8725/chess/whole_lifted/500kb_250kb_25/NPB-Omer/mode1_filtered.bedpe
49 /scratch/ak8725/chess/whole_lifted/500kb_250kb_25/NPB-Omer/mode2_filtered.bedpe


In [8]:
#create fasta files with the mode sequences from npb and omer to compare them
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# Paths to genome FASTA files
npb_genome_file = "/scratch/ak8725/NPB_new/NPB.fna"
omer_genome_file = "/scratch/ak8725/genomes/orufi.fna"

# Path to bedpe file
bedpe_file = "/scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/right_filtered.bedpe"

# Output FASTA files
output_npb_file = "/scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/right_npb.fasta"
output_omer_file = "/scratch/ak8725/chess/whole_lifted/500kb_250kb_all_regions/right_oruf.fasta"

# Create dictionaries to store genome sequences
npb_sequences = SeqIO.to_dict(SeqIO.parse(npb_genome_file, "fasta"))
omer_sequences = SeqIO.to_dict(SeqIO.parse(omer_genome_file, "fasta"))

# Open the bedpe file for reading
with open(bedpe_file, "r") as bedpe_fh:

    # Open output files for writing
    with open(output_npb_file, "w") as npb_fh, open(output_omer_file, "w") as omer_fh:

        # Initialize a counter for sequence headers
        seq_counter = 1

        for line in bedpe_fh:
            fields = line.strip().split("\t")
            chr_npb, start_npb, end_npb, chr_omer, start_omer, end_omer = fields[:6]

            # Extract sequences from the dictionaries
            npb_seq = npb_sequences[chr_npb][int(start_npb):int(end_npb)].seq
            omer_seq = omer_sequences[chr_omer][int(start_omer):int(end_omer)].seq

            # Create SeqRecord objects with headers
            npb_record = SeqRecord(npb_seq, id=f"seq{seq_counter}", description=f"Coordinates: {chr_npb}:{start_npb}-{end_npb}")
            omer_record = SeqRecord(omer_seq, id=f"seq{seq_counter}", description=f"Coordinates: {chr_omer}:{start_omer}-{end_omer}")

            # Write SeqRecord objects to the output files
            SeqIO.write(npb_record, npb_fh, "fasta")
            SeqIO.write(omer_record, omer_fh, "fasta")

            # Increment the sequence counter
            seq_counter += 1

print("FASTA sequences extracted and saved to output files.")


FASTA sequences extracted and saved to output files.


### Paiwise alignments of windows with emboss stretcher (global alignment)

<b> Use script stretcher.sh <b>

Output:<br>
similarity_mode1.txt<br>
similarity_mode2.txt

# NLR genes

In [5]:
grep 'NLR' ../annotations/Nipponbare_LRR-CR.gff > ../annotations/NLR.gff

In [9]:
cut -f 2,3,4 ../annotations/domestication_genes.txt > ../annotations/domestication_genes.bed